#Training StyleGAN2 on Colab

Couple of quick nuances of Google Colab:


1.   When referring to a path on your drive and there is spaces in the folder name for example a folder called "My Folder" - you must refer to this as My\ Folder. the "\" indicates the space is not a break in the code
2.   Try not to play around this the code too much, my understanding is vague and I wont be able to problem solve individual issues
3.   Google Colab has issues. It times out, it has a capacity which based on the number of uses can max out and then randomly just disconnect you, so monitor it often
4. You can't close the tab or your computer while performing this training
5. You must mount your drive every time and select tensorflow 1.x every time. Installing the Repo isnt a must every time but if you are getting errors, reinstalling it sometimes helps. Often its just google colab being temperamental.
6. For a video on this whole process to give you a bit more depth of understanding: https://youtu.be/hv3A62Ojqdg

##Mounting Google Drive


First, connect your Drive to Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install the repo


In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

/content/drive/My Drive
mkdir: cannot create directory ‘stylegan2-colab’: File exists
/content/drive/My Drive/stylegan2-colab
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/drive/My Drive/stylegan2-colab/stylegan2
mkdir: cannot create directory ‘pkl’: File exists
/content/drive/My Drive/stylegan2-colab/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2-colab/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:02, 43.2MB/s]
/content/drive/My Drive/stylegan2-colab/stylegan2
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘results/00001-pretrained’: File exists
/content/drive/My Drive/stylegan2-colab/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-colab/stylegan2/results/00001-pretrained/stylegan2-ffhq-config-f.pkl


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Convert from RGBA to RGB
Our .png images are RGBA format - they need to be pure RGB (no transparency).

Only change the Origin_Path and Destination_Path.

Put a # in front of os.makedirs(destination_path) if you have already created the destination folder.

Where we reference a folder path in "" you dont use '\ ' to denote a space

In [ ]:
#Transform RGBA to RGB
import os
from PIL import Image
import random
from tqdm import tqdm
#number of images to pick from original set
origin_path = "/content/drive/My Drive/Faces/African4/"
destination_path = "/content/drive/My Drive/Machine Learning Outputs/African4/"
os.makedirs(destination_path)
file_dir = os.listdir(origin_path)
print(len(file_dir))
picked_imgs = file_dir
for entry in tqdm(picked_imgs, position=0, leave=True):
  img  = Image.open(origin_path+entry)
  img.load() # required for png.split()
  img = img.resize((1024, 1024))
  background = Image.new("RGB", img.size, (196, 255, 196))
  if len(img.split()) != 4:
    img.save(destination_path+entry, 'PNG', quality=100)
    continue
  background.paste(img, mask=img.split()[3]) # 3 is the alpha channel
  background.save(destination_path+entry, 'PNG', quality=100)


  0%|          | 0/2020 [00:00<?, ?it/s]

2020


100%|██████████| 2020/2020 [17:16<00:00,  1.95it/s]


##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords).

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.

In [ ]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python dataset_tool.py create_from_images ./datasets/African4 /content/drive/My\ Drive/Machine\ Learning\ Outputs/African4/

/content/drive/My Drive/stylegan2-colab/stylegan2
Loading images from "/content/drive/My Drive/Machine Learning Outputs/African4/"
100% 2020/2020 [11:30<00:00,  2.93it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset=`

This should be the name you used after ".\datasets\" when converting your dataset.

In [ ]:
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=African4 --mirror-augment=true --metrics=none

/content/drive/My Drive/stylegan2-colab/stylegan2
Local submit - run_dir: results/00015-stylegan2-African4-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x80a8000 @  0x7f7cc4b7b001 0x7f7cc1607765 0x7f7cc166bbb0 0x7f7cc166da4f 0x7f7cc1704048 0x50a635 0x50cd96 0x507d64 0x509042 0x594931 0x549e5f 0x5513d1 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x588e5c 0x59fc4e
tcmalloc: large alloc 4294967296 bytes == 0x7f7afc422000 @  0x7f7cc4b791e7 0x7f7cc16075e1 0x7f7cc166bc78 0x7f7cc166bf37 0x7f7cc1703f28 0x50a635 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64
tcmalloc: large al

In [ ]:
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!bash ./easy-install.sh

/content/drive/My Drive
mkdir: cannot create directory ‘stylegan2-colab’: File exists
/content/drive/My Drive/stylegan2-colab
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/drive/My Drive/stylegan2-colab/stylegan2
     |████████████████████████████████| 102kB 2.5MB/s 
     |████████████████████████████████| 163kB 9.0MB/s 
     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
     |████████████████████████████████| 10.8MB 8.5MB/s 
     |████████████████████████████████| 2.8MB 40.4MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 133kB 50.7MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 17.3MB 195kB/s 
     |███████████████████████

Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python run_generator.py generate-images --network=/content/drive/My\ Drive/Machine\ Learning\ Outputs/African\ Pkls/African4.pkl --seeds=1-1000 --truncation-psi=0.7

/content/drive/My Drive/stylegan2-colab/stylegan2
Local submit - run_dir: results/00017-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/drive/My Drive/Machine Learning Outputs/African Pkls/African4.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 1 (1/1000) ...
Generating image for seed 2 (2/1000) ...
Generating image for seed 3 (3/1000) ...
Generating image for seed 4 (4/1000) ...
Generating image for seed 5 (5/1000) ...
Generating image for seed 6 (6/1000) ...
Generating image for seed 7 (7/1000) ...
Generating image for seed 8 (8/1000) ...
Generating image for seed 9 (9/1000) ...
Generating image for seed 10 (10/1000) ...
Generating image for seed 11 (11/1000) ...
Generating image for seed 12 (12/1000) ...
Generating image for seed 13 (13/1000) ...
Generating image for seed

Lets make the white background transparent and rescale to 250x250

Let’s zip the generated files and download them.

In [ ]:
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!zip -r megapack1024-lightgreen-50k-p5.zip /content/drive/My\ Drive/stylegan2-colab/stylegan2/results/00009-generate-images